# ADAM NOWAK
https://www.geeksforgeeks.org/recommendation-system-in-python/

### CONTENT-BASED RECOMMENDATION SYSTEM 
#### Item profile
1. TF - term frequency -  It displays the regularity with which a certain term or word occurs in a text corpus or document. - IS USED TO RANK TERM in the document - identifying important terms in the document
2. IDF - inverse document frequency - is employed in text analysis and information retrieval to evaluate the significance of phrases within a set of documents. IDF measures how uncommon or unique a term is in the corpus. - common terms has low ITF values, when rare items high. 

We're going to use TF-IDF vectorizer to count the number of times element appear and to measure how significant its role is. 

$TF-IDFscore(w_{ij}) = TF_{ij} * IDF_i$

#### User profile 
User profile is a vector which describes user preferences. 
During the creation of the user’s profile, we use a utility matrix that describes the relationship between user and item. 


Problems which this system contains: 
1. Finding the appropriate feature is hard.
2. Doesn’t recommend items outside the user profile.

## Collaborative Filtering
Collaborative filtering is based on the idea that similar people (based on the data) generally tend to like similar things. It predicts which item a user will like based on the item preferences of other similar users. 

This system uses user-item matrix to generate recommendations. 
This matrix contains the values that indicate a user’s preference towards a given item. These values can represent either explicit feedback (direct user ratings) or implicit feedback (indirect user behavior such as listening, purchasing, watching).
1. explicit feedback - the amount of data collected from the users when they choose to do so (personal ratings). - USER TELLS WHAT he/she LIKES!
2. implicit feedback - we track user behavior to predict their preference. - we try to infer what she/he likes from usage data. 

Problems: 
1. Hard to find new features which improves quality of the model. 
2. cannot handle fresh item - cold start. 


### General Information: 
#### Problems: 
1. Cold start problem. 
2. evolving with time. 
3. scalability 

#### Approaches to build such a system: 
1. Popularity 
2. Classification model - based on some features like (purchase history or product info classifier can count the probability that the user will buy some product) - less efficient than collaborative filtering methods. 
3. co-occurrence 
4. matrix factorization 




# Book recommendation system

In [2]:
import numpy as np
import pandas as pd

books_df = pd.read_csv('Books.csv')
ratings_df = pd.read_csv('Ratings.csv')
users_df = pd.read_csv('Users.csv')

/var/folders/1_/_d0g9n0d6qg8j72vvxsrnksw0000gn/T/ipykernel_44169/2779536466.py:4: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books_df = pd.read_csv('Books.csv')


In [3]:
books_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271360 non-null  object
 1   Book-Title           271360 non-null  object
 2   Book-Author          271358 non-null  object
 3   Year-Of-Publication  271360 non-null  object
 4   Publisher            271358 non-null  object
 5   Image-URL-S          271360 non-null  object
 6   Image-URL-M          271360 non-null  object
 7   Image-URL-L          271357 non-null  object
dtypes: object(8)
memory usage: 16.6+ MB


In [4]:
books_df = books_df[['ISBN', 'Book-Title', 'Book-Author']]
books_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   ISBN         271360 non-null  object
 1   Book-Title   271360 non-null  object
 2   Book-Author  271358 non-null  object
dtypes: object(3)
memory usage: 6.2+ MB


In [5]:
books_df.dropna(inplace=True)

In [6]:
books_df.shape

(271358, 3)

In [7]:
ratings_df.shape

(1149780, 3)

In [8]:
ratings_df = ratings_df.dropna(subset=['ISBN', 'Book-Rating'])

In [9]:
#merging datasets and dropping useless columns 
book_user_rating = books_df.merge(ratings_df, on='ISBN', how='inner')
book_user_rating = book_user_rating[['ISBN', 'Book-Title', 'Book-Author', 'User-ID', 'Book-Rating']]

book_user_rating.head(10)

,ISBN,Book-Title,Book-Author,User-ID,Book-Rating
0,0195153448,Classical Mythology,Mark P. O. Morford,2,0
1,0002005018,Clara Callan,Richard Bruce Wright,8,5
2,0002005018,Clara Callan,Richard Bruce Wright,11400,0
3,0002005018,Clara Callan,Richard Bruce Wright,11676,8
4,0002005018,Clara Callan,Richard Bruce Wright,41385,0
5,0002005018,Clara Callan,Richard Bruce Wright,67544,8
6,0002005018,Clara Callan,Richard Bruce Wright,85526,0
7,0002005018,Clara Callan,Richard Bruce Wright,96054,0
8,0002005018,Clara Callan,Richard Bruce Wright,116866,9
9,0002005018,Clara Callan,Richard Bruce Wright,123629,9


In [10]:
users_df.head(10)

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN
5,6,"santa monica, california, usa",61.0
6,7,"washington, dc, usa",NaN
7,8,"timmins, ontario, canada",NaN
8,9,"germantown, tennessee, usa",NaN
9,10,"albacete, wisconsin, spain",26.0


In [11]:
print(f"Unique users: {ratings_df['User-ID'].nunique()}")
print(f"Unique books: {ratings_df['ISBN'].nunique()}")

Unique users: 105283
Unique books: 340556


In [12]:
#list of some books
selected_books = ["Where the Heart Is (Oprah's Book Club (Paperback))",
                  "The Surgeon",
                  "I Know This Much Is True"]

filtered_books = book_user_rating[book_user_rating['Book-Title'].isin(selected_books)]

#gruop by title and count reviews 
reviews_count = filtered_books.groupby('Book-Title')['Book-Rating'].count()

print(reviews_count)

Book-Title
I Know This Much Is True                              276
The Surgeon                                           175
Where the Heart Is (Oprah's Book Club (Paperback))    585
Name: Book-Rating, dtype: int64


In [13]:
book_user_rating.shape

(1031134, 5)

#### Only books with more than 100 reviews and users with more than 20 reviews 

In [14]:
book_counts = book_user_rating['Book-Title'].value_counts()
popular_books = book_counts[book_counts > 100].index
filtered_books = book_user_rating[book_user_rating['Book-Title'].isin(popular_books)]
filtered_books.shape

(182799, 5)

In [15]:
user_counts = filtered_books['User-ID'].value_counts()
active_users = user_counts[user_counts > 20].index
filtered_data = filtered_books[filtered_books['User-ID'].isin(active_users)]
filtered_data.shape

(92883, 5)

In [16]:
prepared_data = filtered_data.sort_values(by=['Book-Title', 'User-ID'])
prepared_data.shape

(92883, 5)

In [17]:
prepared_data.head(5)

,ISBN,Book-Title,Book-Author,User-ID,Book-Rating
94366,0451524934,1984,George Orwell,254,9
240083,0451519841,1984,George Orwell,7346,8
94372,0451524934,1984,George Orwell,11676,0
240085,0451519841,1984,George Orwell,11676,0
307406,0452262933,1984,George Orwell,11676,10


In [18]:
pivot_table = prepared_data.pivot_table(
    index='Book-Title',        
    columns='User-ID',   
    values='Book-Rating',   
    fill_value=0            
)

pivot_table.head()

User-ID,243,254,507,638,882,1131,1435,1848,1903,2033,...,276231,276463,276680,277195,277427,277639,278144,278418,278535,278633
Book-Title,,,,,,,,,,,,,,,,,,,,,
1984,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0
24 Hours,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Books in rows, users in columns! 

In [19]:
from sklearn.neighbors import NearestNeighbors

In [20]:
matrix = pivot_table.values
#training model with cosine metric
knn_model = NearestNeighbors(metric='cosine', algorithm='auto')
knn_model.fit(matrix)

NearestNeighbors(metric='cosine')

In [21]:
distances, indices = knn_model.kneighbors(matrix, n_neighbors=5)

In [22]:
pivot_table.iloc[indices[6]].index.values

array(['A Beautiful Mind: The Life of Mathematical Genius and Nobel Laureate John Nash',
       "Schindler's List", 'Plum Island', 'Four Blondes',
       'Chasing the Dime'], dtype=object)

In [23]:
def book_recommendations(book_title):
    try:
        book = pivot_table.loc[book_title]
        
        distances, indices = knn_model.kneighbors([book.values], n_neighbors=6)
        
        recommended_books = pd.DataFrame({
            'title':  pivot_table.iloc[indices[0]].index.values,
            'distance': distances[0],
        }).sort_values(by="distance", ascending=False)
        
        return recommended_books
    except Exception as e:
        return str(e)
        
book_title = "The Catcher in the Rye"  
recommended_books = book_recommendations(book_title)
recommended_books

,title,distance
5,Lord of the Flies,0.822362
4,Catch 22,0.822274
3,Brave New World,0.820174
2,To Kill a Mockingbird,0.809455
1,The Great Gatsby,0.792062
0,The Catcher in the Rye,0.000000


In [59]:
book_recommendations("The Surgeon")

,title,distance
5,Seven Up (A Stephanie Plum Novel),7.700975e-01
4,The Mulberry Tree,7.616649e-01
3,Hard Eight : A Stephanie Plum Novel (A Stephan...,7.537151e-01
2,Mercy,7.407656e-01
1,"Forever... : A Novel of Good and Evil, Love an...",7.300370e-01
0,The Surgeon,1.110223e-16


In [24]:
book_title = "1984"
recommended_books = book_recommendations(book_title)
recommended_books

,title,distance
5,"The Vampire Lestat (Vampire Chronicles, Book II)",8.041461e-01
4,Slaughterhouse Five or the Children's Crusade:...,8.018863e-01
3,The Handmaid's Tale,7.950270e-01
2,Brave New World,7.914254e-01
1,Animal Farm,7.287061e-01
0,1984,1.110223e-16


#### Results 
Distance in cosine metric is a value from 0 to 1, where 0 means no similarities and 1 means full agreement. On the example of book called "1984" we can observe that our system is working correctly returning 5 books with distance close to 1, so they are pretty similar to the "1984" in the aspect of RATING value. We can see that even "Animal Farm" is on the list with btw is another George Orwell book ;))

### CSR matrix method on full book_user_ratings matrix 

In [41]:
from scipy.sparse import csr_matrix

#filtering data
book_counts= book_user_rating['Book-Title'].value_counts()
popular_books = book_counts[book_counts > 10].index
filtered_books = book_user_rating[book_user_rating['Book-Title'].isin(popular_books)]

user_counts = filtered_books['User-ID'].value_counts()
active_users = user_counts[user_counts > 5].index
filtered_data = filtered_books[filtered_books['User-ID'].isin(active_users)]

prepared_data_csr = filtered_data.sort_values(by=['Book-Title', 'User-ID'])

#mapping because its easier to use 0,1,2... values then ISBN numbers or user ids

user_id_mapping = {user_id: idx for idx, user_id in enumerate(prepared_data_csr['User-ID'].unique())}
isbn_mapping = {isbn: idx for idx, isbn in enumerate(prepared_data_csr['ISBN'].unique())}

book_user_rating_csr = prepared_data_csr.copy()

#mapping to used is and isbn to numeric values
book_user_rating_csr['user_idx'] = book_user_rating_csr['User-ID'].map(user_id_mapping)
book_user_rating_csr['isbn_idx'] = book_user_rating_csr['ISBN'].map(isbn_mapping)

rows = book_user_rating_csr['user_idx'].values
cols = book_user_rating_csr['isbn_idx'].values
data = book_user_rating_csr['Book-Rating'].values

#creating csr matrix 
csr_matrix_data = csr_matrix((data, (rows, cols)), shape=(len(user_id_mapping), len(isbn_mapping)))

#basic matrix info 
print(f"Shape of the matrix: {csr_matrix_data.shape}")
print(f"Number of non-zero elements: {csr_matrix_data.nnz}")


Shape of the matrix: (11953, 29036)
Number of non-zero elements: 463418


Calculating similarities between books and searching for recommendation based on one book. Item-based Collaborative Filtering

In [65]:
from sklearn.neighbors import NearestNeighbors

knn_csr_model = NearestNeighbors(metric='cosine', algorithm='auto', n_neighbors=6) 
knn_csr_model.fit(csr_matrix_data.T)  

NearestNeighbors(metric='cosine', n_neighbors=6)

In [61]:
distances, indices = knn_csr_model.kneighbors(csr_matrix_data.T[0], n_neighbors=6)

for i, (idx, distance) in enumerate(zip(indices.flatten(), distances.flatten())):
    print(f"{i+1}. Book Index: {idx}, Distance: {distance}")

1. Book Index: 0, Distance: 0.0
2. Book Index: 11781, Distance: 0.4021654953349484
3. Book Index: 8744, Distance: 0.40357627844090105
4. Book Index: 18317, Distance: 0.42829025984821
5. Book Index: 14178, Distance: 0.4319983689733131
6. Book Index: 14233, Distance: 0.4321229678189916


In [62]:
def book_recommendations_csr_matrix(book_title):
    try:
        #finding ISBN value based on title
        book_isbn = books_df.loc[books_df['Book-Title'] == book_title, 'ISBN']
        if book_isbn.empty:
            return "Book not found in the dataset."
        book_isbn = book_isbn.values[0]
        
        #get book index in mapping
        book_idx = isbn_mapping.get(book_isbn)
        if book_idx is None:
            return "Book not found in the dataset."

        distances, indices = knn_csr_model.kneighbors(csr_matrix_data.T[book_idx], n_neighbors=6)
        
        #creating a list of recommendations
        rec_books = []
        for idx, distance in zip(indices.flatten(), distances.flatten()):
            if idx != book_idx:  
                similar_isbn = list(isbn_mapping.keys())[list(isbn_mapping.values()).index(idx)]
                similar_book_title = books_df.loc[books_df['ISBN'] == similar_isbn, 'Book-Title'].values[0]
                rec_books.append({'title': similar_book_title, 'distance': distance})
        
        #converting a list to dataframe for cooler display
        recommended_books = pd.DataFrame(rec_books).sort_values(by="distance", ascending=False)
        
        return recommended_books

    except Exception as e:
        return str(e)

book_recommendations_csr_matrix("1984")

,title,distance
4,Far from the Madding Crowd,0.842614
3,S.,0.837060
2,We,0.827236
1,The Cigarette Girl: A Novel,0.826642
0,Animal Farm,0.817231


In [63]:
book_recommendations_csr_matrix("The Surgeon")

,title,distance
4,Premeditated Marriage,0.787090
0,Wildest Hearts,0.766365
1,Priceless,0.766365
2,Second Chances,0.766365
3,Impetuous,0.766365


### Results knn and CSR matrix
The difference here is that I used csr matrix == sparse matrix, so in this case matrix contains only not null values, where in the pivot_table we filled every null with 0. CSR matrix allows me to analyse much more examples of books and users, because on the lower complexity. CSR matrix is much more efficient and is scaling up way better in comparison with typical pivot table, which is a full matrix . 